In [1]:
import numpy as np 
import pandas as pd

from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [2]:
data=pd.read_csv('../input/customerexp/improve.csv',nrows=50)
data.head()

,Date,Improve,Super,ImpCrit
0,2011-10-01,Get treatment I requested.,Care/ Treatment,1.0
1,2011-10-01,Provide Gym staff to get Gym up and running!,Care/ Treatment,1.0
2,2011-10-01,To look at the problems properly and give you ...,Care/ Treatment,2.0
3,2011-10-01,To be more involved.,Involvement,2.0
4,2011-10-01,My drug problem was with Subutex. I was given...,Care/ Treatment,2.0


In [3]:
datafinal=data.dropna(axis=0)
comment_columns = 'Improve'
target_columns = datafinal.columns[2:]
y = datafinal[target_columns]
X = datafinal[comment_columns]
y['ImpCrit'].values[y['ImpCrit'].values > 4] = 5
y['ImpCrit']=y['ImpCrit'].abs()
Y1=pd.get_dummies(y['Super'],drop_first = True)
Y2=pd.get_dummies(y['ImpCrit'],drop_first = True)
Y=pd.concat([Y1,Y2],axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [4]:
len(Y.columns)

5

In [5]:
import string

def clean(instr):
    return instr.translate(str.maketrans('', '', string.punctuation))
X = X.apply(clean) 
X = X.replace(r'\n',  ' ', regex=True) 

In [6]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

In [7]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [8]:
import tokenization

In [9]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [7]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(5, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [11]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [12]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [13]:
X

0                             Get treatment I requested
1           Provide Gym staff to get Gym up and running
2     To look at the problems properly and give you ...
3                                   To be more involved
4     My drug problem was with Subutex  I was given ...
5     The prisoner would be able to choose what trea...
6     If people would like to be put on medication f...
7     Nothing really only the gym and need of staff ...
8     Medication  What we get on the outside ie Zopi...
9     I would have liked for the IDTS Doctor to have...
10                                 Time to get our meth
11    Getting back on to Methadone programme can tak...
12                              To have longer sessions
13                  Access to the doctor without triage
14    To come right away not leaving it about 3 or 4...
15    I should have gone to the MSU and am still in ...
17        Noone likes giving benzos and thats not right
18    I would like to have a support group so we

In [14]:
train_input = bert_encode(X.values, tokenizer, max_len=160)
train_labels = Y.values

In [15]:
model = build_model(bert_layer, max_len=160)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]      

In [16]:
train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=3,
    batch_size=16
)

Epoch 1/3
2/2 [==============================] - 21s 10s/step - loss: 2.2349 - accuracy: 0.1000 - val_loss: 2.2235 - val_accuracy: 0.0000e+00
Epoch 2/3
2/2 [==============================] - 17s 8s/step - loss: 2.0613 - accuracy: 0.1500 - val_loss: 2.1839 - val_accuracy: 0.0000e+00
Epoch 3/3
2/2 [==============================] - 17s 8s/step - loss: 1.8989 - accuracy: 0.3000 - val_loss: 2.1422 - val_accuracy: 0.0000e+00


In [28]:
from keras.utils.np_utils import to_categorical
predictions = model.predict(train_input[:5])


In [38]:
predictions.round().astype(int)

array([[1, 0, 1, 1, 0],
       [1, 0, 1, 0, 0],
       [1, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0],
       [0, 1, 1, 0, 1],
       [1, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1],
       [0, 0, 1, 0, 1],
       [0, 0, 1, 1, 0],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 1, 1, 1, 0],
       [0, 0, 0, 1, 0],
       [1, 1, 1, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 1, 0, 0],
       [1, 0, 1, 0, 0],
       [1, 0, 1, 1, 0],
       [1, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0]])